In [1]:
#Import all the dependencies
import sys
import pandas as pd
from gensim.models.doc2vec import Doc2Vec
sys.path.append("../") # go to parent dir
from src.data.diversity_document import DiversityDocument
from src.data.diversity_corpus import DiversityUtils

df = pd.read_csv('../data/external/2017-18stats.csv')
df['CompanyLinkToGPGInfo'] = df['CompanyLinkToGPGInfo'].astype(str)

In [2]:
#split the corpus into quartiles by the "best" companies on diversity numbers. 
upper_q = df.DiffMeanHourlyPercent.quantile(0.75)
lower_q = df.DiffMeanHourlyPercent.quantile(0.25)
df_bad = df.loc[df.DiffMeanHourlyPercent > upper_q]
df_good = df.loc[df.DiffMeanHourlyPercent < lower_q]
print('Number of companies in worst quarter: %s' % df_bad.DiffMeanHourlyPercent.count())
print('Number of companies in best quarter: %s' % df_good.DiffMeanHourlyPercent.count())

Number of companies in worst quarter: 2513
Number of companies in best quarter: 2537


In [3]:
class LabeledLineSentence(object):
    def __init__(self, df):
       self.df = df
    def __iter__(self):
        for label, doc in DiversityUtils.iter_diversity_data(self.df):
            yield TaggedDocument(words=doc,tags=list(label))


In [4]:
bad_docs = LabeledLineSentence(df)
bad_docs = list(bad_docs)
print(bad_docs)

ValueError: too many values to unpack (expected 2)

In [5]:
model = Doc2Vec(vector_size=300, window=10, min_count=3, workers=11,alpha=0.025, min_alpha=0.025) # use fixed learning rate
model.build_vocab(bad_docs)
model.train(bad_docs, total_examples=model.corpus_count, epochs=model.epochs)


ValueError: too many values to unpack (expected 2)

In [6]:
similar_doc = model.wv.most_similar('male')
#print(similar_doc)
#print(similar_doc)
#print (model.docvecs.most_similar(16))
tokens = "a new sentence to match".split()
new_vector = model.infer_vector(tokens)
sims = model.docvecs.most_similar('1ba471c85aff746b9fa899572ef48eee', topn=5)
print(model.epochs)
#print(model)

TypeError: unsupported operand type(s) for ** or pow(): 'list' and 'int'